In [1]:
#here we import our libraries
import pandas as pd
import numpy as np
import os

# Data Wrangling

In [2]:
#and now our data, starting with defining the first part of the path
path = r'C:\Users\Admin\Documents\Dec 22 Instacart Analysis'

In [3]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
#let's see what we're working with
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df.shape

(206209, 10)

In [6]:
df.isnull().sum()

user_id             0
First Name      11259
Surnam              0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [7]:
df_dups = df[df.duplicated()]

In [8]:
df_dups

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


In [5]:
#I wanted to see if the date joined was the same for all customers like it was for Rockbuster. If so we could drop it
df['date_joined'].value_counts()

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

For the time being I don't want to drop any columns since I don't know what is going to be needed in the analysis just yet. I will fix a few names, however. 

In [9]:
df.rename(columns = {'n_dependants' : 'n_dependents'}, inplace = True)

In [10]:
df.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [11]:
df.rename(columns = {'income' : 'annual_income'}, inplace = True)

In [9]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependents,marital_status,annual_income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


much better!

In [12]:
#on second thought, for consistency's sake and I like how it reads better, I want to make everything the same format
df.rename(columns = {'First Name' : 'first_name'}, inplace = True)
df.rename(columns = {'Surnam' : 'last_name'}, inplace = True)
df.rename(columns = {'Gender' : 'gender'}, inplace = True)
df.rename(columns = {'STATE' : 'state'}, inplace = True)
df.rename(columns = {'Age' : 'age'}, inplace = True)

In [11]:
df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependents,marital_status,annual_income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


I know it's not necessary but I like this so much better

# Quality Checks

In [12]:
df.describe()

,user_id,age,n_dependents,annual_income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


I want to know who is making 594k annually and what they do for a living

At a glance, none of this looks outwardly problematic, so that's nice

In [13]:
#here is the code that checks for mixed data types
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

first_name


In [13]:
#everything in the first_name column should be a string
df['first_name'] = df['first_name'].astype('str')

In [15]:
#looking for missing values
df.isnull().sum()

user_id           0
first_name        0
last_name         0
gender            0
state             0
age               0
date_joined       0
n_dependents      0
marital_status    0
annual_income     0
dtype: int64

yay!

In [16]:
#finally, we check for duplicates
df_dups = df[df.duplicated()]

In [17]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependents,marital_status,annual_income


we are lucky lucky ducks today!

# Merging data

In [14]:
#first, we have to import our ords/prods data set
df2 = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_flagged.pkl'))

In [19]:
pd.options.display.max_columns = None

In [20]:
df2.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,order_cost,avg_spent_per_item,spender_type,order_frequency,frequency_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,195,Soda,77,7,9,both,Mid-range product,Regularly busy,Average orders,10,New customer,30,6.084746,Low spender,20.5,Non frequent customer
1,1,2398795,1,2,3,7,15.0,196,1,1,195,Soda,77,7,9,both,Mid-range product,Least busy days,Average orders,10,New customer,34,6.084746,Low spender,20.5,Non frequent customer
2,2,473747,1,3,3,12,21.0,196,1,1,195,Soda,77,7,9,both,Mid-range product,Least busy days,Most orders,10,New customer,27,6.084746,Low spender,20.5,Non frequent customer
3,3,2254736,1,4,4,7,29.0,196,1,1,195,Soda,77,7,9,both,Mid-range product,Least busy days,Average orders,10,New customer,25,6.084746,Low spender,20.5,Non frequent customer
4,4,431534,1,5,4,15,28.0,196,1,1,195,Soda,77,7,9,both,Mid-range product,Least busy days,Most orders,10,New customer,55,6.084746,Low spender,20.5,Non frequent customer


In [21]:
#the merge column needs to be dropped or it will cause problems when we do another merge
df2 = df2.drop(columns = ['_merge'])

In [22]:
#checking the shapes
df.shape

(206209, 10)

In [23]:
df2.shape

(32404859, 25)

In [24]:
df2['user_id'].value_counts()

201268    3704
129928    3637
164055    3061
186704    2936
176478    2921
          ... 
188345       3
70320        3
203875       2
124615       2
91567        1
Name: user_id, Length: 206209, dtype: int64

So there are the same number of user IDs in both dataframes, which is what we want

In [25]:
#now we do the merging
df_merged = df.merge(df2, on = 'user_id', indicator = True)

In [26]:
df_merged.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependents,marital_status,annual_income,Unnamed: 0_x,order_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,order_cost,avg_spent_per_item,spender_type,order_frequency,frequency_flag,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,443895,2543867,5,1,9,30.0,196,2,0,195,Soda,77,7,9,Mid-range product,Busiest days,Average orders,8,New customer,17,7.666667,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,443897,1285508,7,5,15,11.0,196,1,1,195,Soda,77,7,9,Mid-range product,Regularly busy,Most orders,8,New customer,25,7.666667,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,443898,2578584,8,1,15,10.0,196,2,1,195,Soda,77,7,9,Mid-range product,Busiest days,Most orders,8,New customer,25,7.666667,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,443892,423547,2,2,9,14.0,38928,1,0,38932,0% Greek Strained Yogurt,120,16,12,Mid-range product,Regularly busy,Average orders,8,New customer,12,7.666667,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,443893,2524893,3,3,11,30.0,38928,1,1,38932,0% Greek Strained Yogurt,120,16,12,Mid-range product,Least busy days,Most orders,8,New customer,23,7.666667,Low spender,19.0,Regular customer,both


In [27]:
#before I save this, I want to drop the merge column again since it's not necessary
df_merged = df_merged.drop(columns = ['_merge'])

# finally, we export

In [28]:
#this madness definitely needs to be a pickle
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_custs.pkl'))